In [101]:
# Part 1. Import Python Modules

from selenium import webdriver
from time import sleep
from urllib.parse import urlparse
from urllib.parse import parse_qs
import csv
from urllib.request import urlopen
import shutil
import os.path
import datetime

period_end = datetime.datetime(2017, 10, 4)


def unix_time_millis(dt):
    dt += datetime.timedelta(days=1)
    return str(int((dt - datetime.datetime.utcfromtimestamp(0)).total_seconds()))

period_end = unix_time_millis(period_end)


# Part 2. Download Nasdaq Tickers

url = "http://www.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download"
file_name = "nasdaq.csv"
with urlopen(url) as response, open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

# Part 3. Set up Chromedriver

options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": r"yahoo_data",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
driver = webdriver.Chrome(chrome_options=options)
crumb = None

# Part 4. Get Crumb Value

def get_crumb():
    driver.get("https://finance.yahoo.com/quote/AAPL/"
               "history?period1=0&period2=2000000000&interval=1d&filter=history&frequency=1d")
    sleep(10)
    el = driver.find_element_by_xpath("// a[. // span[text() = 'Download Data']]")
    link = el.get_attribute("href")
    a = urlparse(link)
    crumb = parse_qs(a.query)["crumb"][0]
    return crumb

for i in range(10):
    if not crumb:
        crumb = get_crumb()
    else:
        break

if not crumb:
    raise ValueError("Crumb Not Set")


# Part 5. Download Yahoo Quotes

for file in os.listdir("yahoo_data"):
    file_path = os.path.join("yahoo_data", file)
    os.unlink(file_path)


def download_yahoo_quotes(ticker):
    driver.get("https://query1.finance.yahoo.com/v7/finance/download/"
               "{0}?period1=0&period2={1}&interval=1d&events=history&crumb={2}".format(ticker, period_end, crumb))
    sleep(1)

for t in range(3):
    with open(file_name) as tickers_file:
        tickers_reader = csv.reader(tickers_file)
        next(tickers_reader)
        for line in tickers_reader:
            ticker = line[0].strip()
            if not os.path.isfile("./yahoo_data/{}.csv".format(ticker)):
                download_yahoo_quotes(ticker)

# Part 6. Identify Missing Data

errors = {}
with open(file_name) as tickers_file:
    tickers_reader = csv.reader(tickers_file)
    next(tickers_reader)
    for line in tickers_reader:
        ticker = line[0].strip()
        if not os.path.isfile("./yahoo_data/{}.csv".format(ticker)):
            errors[ticker] = "Quotes not downloaded."

with open("scraping_errors.csv", "w+") as errors_file:
    errors_writer = csv.writer(errors_file)
    errors_writer.writerow(["ticker", "error"])
    for ticker in errors:
        errors_writer.writerow([ticker, errors[ticker]])

C:\Users\nicko\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: use options instead of chrome_options


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [17]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup 
import csv 
import bs4 as bs
import urllib.request

### Web Scrape | Currencies | Current Prices

In [41]:
names=[]
prices=[]
changes=[]
percentChanges=[]
marketCaps=[]
totalVolumes=[]
circulatingSupplys=[]
 
CurrenciesUrl = "https://in.finance.yahoo.com/currencies"
r= requests.get(CurrenciesUrl)
data=r.text
soup=BeautifulSoup(data)
 
counter = 40
for i in range(40, 404, 14):
   for listing in soup.find_all('tr', attrs={'data-reactid':i}):
      for name in listing.find_all('td', attrs={'data-reactid':i+3}):
         names.append(name.text)
      for price in listing.find_all('td', attrs={'data-reactid':i+4}):
         prices.append(price.text)
      for change in listing.find_all('td', attrs={'data-reactid':i+5}):
         changes.append(change.text)
      for percentChange in listing.find_all('td', attrs={'data-reactid':i+7}):
         percentChanges.append(percentChange.text)
            
currencies = pd.DataFrame({"Names": names, 
              "Prices": prices, 
              "Change": changes, 
              "% Change": percentChanges})
currencies


,Names,Prices,Change,% Change
0,USD/INR,71.2790,-0.2200,-0.3077%
1,EUR/INR,77.8032,-0.4664,-0.60%
2,GBP/INR,92.0338,-0.0599,-0.07%
3,AED/INR,19.3630,-0.0600,-0.31%
4,INR/JPY,1.5369,+0.0045,+0.29%
5,SGD/INR,51.3090,-0.1300,-0.25%
6,USD/IDR,"13,704.0000",+61.0000,+0.4471%
7,USD/THB,31.2900,-0.0100,-0.0319%
8,USD/MYR,4.1450,+0.0045,+0.1087%
9,USD/ZAR,14.9572,-0.1000,-0.6641%


In [94]:
import bs4 as bs
import urllib.request
import pandas as pd
import time
  
def get_ticker(ticker, day_one, day_two):
      
    url = 'https://finance.yahoo.com/quote/' + ticker + '/history?period1=' + day_one + '&period2=' + day_two + '&interval=1d&filter=history&frequency=1d'
    source = urllib.request.urlopen(url).read()      
    soup =bs.BeautifulSoup(source,'lxml')
    tr = soup.find_all('tr')
      
    data = []
      
    for table in tr:
        td = table.find_all('td')
        row = [i.text for i in td]
        data.append(row)        
      
    columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
  
    data = data[1:-2]
    df = pd.DataFrame(data)
    df.columns = columns
    df.set_index(columns[0], inplace=True)   
    df = df.iloc[::-1]
    df.dropna(inplace=True)
      
    return df
     
 
# April 3, 2018 = 1522728000  (seconds since UNIX epoch in 1970)
# June 12, 2018 = 1528776000
# https://finance.yahoo.com/quote/AAPL/history?period1=1522728000&period2=1528776000&interval=1d&filter=history&frequency=1d
 
 
format_string='%Y-%m-%d %H:%M:%S'
 
# One day (86400 second) adjustment required to get dates printed to match web site manual output
date1='2017-04-03 00:00:00'
date1_epoch = str(int(time.mktime(time.strptime(date1,format_string)))- 86400)
print("")
print(date1, date1_epoch)
 
date2='2019-06-12 00:00:00'
date2_epoch = str(int(time.mktime(time.strptime(date2,format_string))))
print(date2, date2_epoch)
 
df = get_ticker('AAPL', date1_epoch, date2_epoch)
df


2017-04-03 00:00:00 1491105600
2019-06-12 00:00:00 1560312000


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
"Jan 24, 2019",154.11,154.48,151.74,152.70,150.08,"25,441,500"
"Jan 25, 2019",155.48,158.13,154.32,157.76,155.06,"33,535,500"
"Jan 28, 2019",155.79,156.33,153.66,156.30,153.62,"26,192,100"
"Jan 29, 2019",156.25,158.13,154.11,154.68,152.03,"41,587,200"
"Jan 30, 2019",163.25,166.15,160.23,165.25,162.42,"61,109,800"
...,...,...,...,...,...,...
"Jun 06, 2019",183.08,185.47,182.15,185.22,183.53,"22,526,300"
"Jun 07, 2019",186.51,191.92,185.77,190.15,188.42,"30,684,400"
"Jun 10, 2019",191.81,195.37,191.62,192.58,190.82,"26,220,900"


In [96]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
"Jan 24, 2019",154.11,154.48,151.74,152.70,150.08,"25,441,500"
"Jan 25, 2019",155.48,158.13,154.32,157.76,155.06,"33,535,500"
"Jan 28, 2019",155.79,156.33,153.66,156.30,153.62,"26,192,100"
"Jan 29, 2019",156.25,158.13,154.11,154.68,152.03,"41,587,200"
"Jan 30, 2019",163.25,166.15,160.23,165.25,162.42,"61,109,800"
